In [47]:
import pandas as pd
from mmrotate.structures.bbox import rbox2qbox
import torch
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
import os
from tqdm import tqdm

In [48]:
PATH_BBOXES_DF = '/home/jose/Programas/OCDDataset/detection_outputs/consolidated_df.csv'
BEST_DETECTOR = 'oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped'
GROUND_TRUTH = 'ground-truth'
PATH_OUTPUT = '/home/jose/Programas/OCDDataset/detection_outputs/best_detector_comparison'
PATH_IMAGES = '/home/jose/Programas/OCDDataset/mmrotate-1.x/data/OCD/test/images'

In [49]:
bboxes_df = pd.read_csv(PATH_BBOXES_DF)
bboxes_df = bboxes_df.loc[bboxes_df['evaluator'].isin([BEST_DETECTOR, GROUND_TRUTH])]
assert len(bboxes_df[bboxes_df['evaluator'] == BEST_DETECTOR]) > 0
assert len(bboxes_df[bboxes_df['evaluator'] == GROUND_TRUTH]) > 0

bboxes_df

,file_name,evaluator,cx,cy,width,height,angle,detection_threshold,type
0,exp_M4_Mauricio_mcr5_frame_40,ground-truth,29.50000,247.000000,96.000000,59.000000,1.570796,1.000000,test
1,exp_M4_Mauricio_mcr5_frame_40,ground-truth,104.39654,581.258670,122.377594,83.773506,1.165904,1.000000,test
2,exp_M4_Mauricio_mcr5_frame_40,ground-truth,196.55138,641.218100,88.093500,149.375370,1.098167,1.000000,test
3,exp_M4_Mauricio_mcr5_frame_40,ground-truth,399.00006,710.000060,143.676730,108.300520,0.501013,1.000000,test
4,exp_M4_Mauricio_mcr5_frame_40,ground-truth,233.97860,826.720150,88.457920,144.678960,0.096918,1.000000,test
...,...,...,...,...,...,...,...,...,...
10721,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped,1110.20120,391.458100,16.934748,27.918470,0.574780,0.050564,test
10722,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped,118.35654,36.957123,23.931404,25.671124,0.313665,0.050382,test
10723,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped,652.54205,683.801400,52.167930,27.370080,1.374631,0.050317,test
10724,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped,421.77640,702.954040,38.902233,18.887833,0.342907,0.050263,test


In [57]:
def getBBoxesFromDF(df, evaluator: str):
    return rbox2qbox(
        torch.Tensor(
            df.loc[(df['evaluator'] == evaluator) & (df['detection_threshold'] >= 0.5)][['cx','cy','width','height','angle']].values)
    )


for image_name, image_group in tqdm(bboxes_df.groupby('file_name')):
    path_input_image = os.path.join(PATH_IMAGES, image_name + '.png')
    path_output_image = os.path.join(PATH_OUTPUT, image_name + '_output.png')

    gt_bboxes = getBBoxesFromDF(image_group, GROUND_TRUTH)
    pred_bboxes = getBBoxesFromDF(image_group, BEST_DETECTOR)

    img = Image.open(path_input_image)
    draw = ImageDraw.Draw(img)
    
    for bbox in gt_bboxes:
        draw.polygon(bbox.tolist(), outline="green", width=2)
    for bbox in pred_bboxes:
        draw.polygon(bbox.tolist(), outline="blue", width=2)
    
    img.save(path_output_image)



100%|██████████| 30/30 [00:11<00:00,  2.67it/s]
